****4 Grams of Bytes Extractor ****
* Read the EXE as a binary file
* Count the occurences of every 4-grams instance
* Count the frequency of each predecessor and successor

*This is just a dumb piece of code, we didn't give any particular attention to the beauty, the optimization or the conventions of python when writing this notebook. If you have any remarks about that please leave a comment we'd love to enhance our coding skills.*

In [ ]:
from os import listdir
import os.path
import pandas as pd
import json

In [ ]:
#Path to EXE files samples
base_dir = "../input/firstpart/samples/"
#List of exe hashes with status,
#Clear: Non protected exe (packing or encryption)
malware_list = pd.read_csv('../input/malwarelist/malware_list_sample_1.csv', index_col='Hash Of Malware')
bytes_grams = {}
counter = 1
a = 1
for_deletion = []
for f in listdir(base_dir):
    if not f.endswith('.json') and malware_list.loc[f]['Status']=='clear':
        bytes_grams[f] = {}
        maxfr = 0
        maxfrbg = None
        with open(os.path.join(base_dir,f),'rb') as bf:
            four_grams = bf.read(4).hex()
            bytes_grams[f][four_grams]={
                'frequency' : 1,
                'preds' : {},
                'sucs' : {},
            }
            pred = four_grams
            count_grams = 0
            while four_grams != '':
                count_grams += 1
                four_grams = bf.read(4).hex()
                if four_grams in bytes_grams[f].keys():
                    bytes_grams[f][four_grams]['frequency']+=1
                else:
                    bytes_grams[f][four_grams]={
                        'frequency' : 1,
                        'preds' : {},
                        'sucs' : {}, 
                    }
                if pred in bytes_grams[f][four_grams]['preds'].keys():
                    bytes_grams[f][four_grams]['preds'][pred] += 1
                else :
                    bytes_grams[f][four_grams]['preds'][pred] = 1
                if four_grams in bytes_grams[f][pred]['sucs'].keys():
                    bytes_grams[f][pred]['sucs'][four_grams] += 1
                else :
                    bytes_grams[f][pred]['sucs'][four_grams] = 1
                pred = four_grams
            #Reducing the number of considered 4-grams
            mean_quantile = count_grams/len(bytes_grams[f]) * 2
            for bg in bytes_grams[f].keys():
                if bytes_grams[f][bg]['frequency'] < mean_quantile:
                    for_deletion.append(bg)
                else:
                    temp={k: v for k, v in sorted(bytes_grams[f][bg]['preds'].items(), key=lambda item: item[1])}
                    if temp != {}:
                        keyvalue = list(temp.items())[-1]
                        bytes_grams[f][bg]['preds'] = keyvalue
            for bg in for_deletion:
                del bytes_grams[f][bg]
            for_deletion = []
            for bg in bytes_grams[f].keys():
                temp={k: v for k, v in sorted(bytes_grams[f][bg]['sucs'].items(), key=lambda item: item[1])}
                if temp != {}:
                    keyvalue = list(temp.items())[-1]
                    bytes_grams[f][bg]['sucs'] = keyvalue
            if counter > a * 500:
                with open('4grams%s.json' %a, 'a') as gr:
                    json.dump(bytes_grams,gr)
                del bytes_grams
                bytes_grams = {}
                a += 1
        print(counter)
        counter+=1

In [ ]:
with open('4grams%s.json' %a, 'a') as gr:
    json.dump(bytes_grams,gr)